In [1]:
!pip install gradio torch torchvision opencv-python pillow numpy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import cv2
from PIL import Image
import gradio as gr


In [6]:
class CSRNet(nn.Module):
    def __init__(self):
        super().__init__()
        vgg = models.vgg16_bn(weights=models.VGG16_BN_Weights.IMAGENET1K_V1)

        # first 33 layers of VGG16-bn
        self.frontend = nn.Sequential(*list(vgg.features.children())[:33])

        # dilated backend
        self.backend = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(512, 256, 3, padding=2, dilation=2), nn.ReLU(inplace=True),
            nn.Conv2d(256, 128, 3, padding=1),            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1,   1)
        )

    def forward(self, x):
        x = self.frontend(x)
        x = self.backend(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.backends.cudnn.benchmark = True

model = CSRNet().to(device)
criterion = nn.MSELoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-4)


Using device: cuda
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


100%|██████████| 528M/528M [00:07<00:00, 74.3MB/s]


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CSRNet().to(device)
model.load_state_dict(
    torch.load(
        "/content/drive/MyDrive/deepvision/checkpoints/partB_best.pth",
        map_location=device
    )
)

model.eval()

print("✅ CSRNet model loaded successfully")


✅ CSRNet model loaded successfully


In [9]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet
        std=[0.229, 0.224, 0.225]
    )
])


In [17]:
THRESHOLD = 6  # you can change this

def predict(frame):
    # Convert webcam frame to PIL
    img = Image.fromarray(frame).convert("RGB")

    # Preprocess
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Inference
    with torch.no_grad():
        density_map = model(img_tensor)
        count = density_map.sum().item()

    count = int(count)


    # Alert logic
    if count > THRESHOLD:
        alert = "🚨 ALERT: Crowd Limit Exceeded!"
    else:
        alert = "✅ Crowd Level Safe"

    # Overlay text on frame
    display_frame = frame.copy()
    cv2.putText(display_frame, f"Count: {count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(display_frame, alert, (20, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    return display_frame, f"👥 Crowd Count: {count}", alert


In [18]:
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(sources=["webcam"], streaming=True),
    outputs=[
        gr.Image(label="Live View"),
        gr.Textbox(label="Crowd Count"),
        gr.Textbox(label="Status")
    ],
    live=True,
    title="Real-Time Crowd Counting (CSRNet)",
    description="Webcam-based crowd counting using CSRNet. Alerts trigger when crowd exceeds threshold."
)

interface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d982adb08b91989a32.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d982adb08b91989a32.gradio.live
